# Description de la composition des clusters

- Calculer l'importance relative des variables
- Décrire chaque cluster (persona) par la moyenne et l'écart type de chaque variable à l'intérieur du cluster
- Pour chaque cluster afficher les variables les plus distinctives (par leur présence ou leur absence)

In [ ]:
# Notebook pour la description de la composition des clusters

# Library ----------------------------------------------------------------
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Charger les données et le résultat du clustering
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds")
df_pilot1_2022 <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_2022.rds")

# Sélectionner les variables utilisées pour le clustering
variablesUsedForClustering <- colnames(df_pilot1_2022)[colnames(df_pilot1_2022) %in% colnames(kmeans_result$centers)] # nolint: line_length_linter.

# Créer un dataframe enrichie avec la variable cluster convertie en facteur
dfUsedForClustering <- df_pilot1_2022
dfUsedForClustering$cluster <- as.factor(kmeans_result$cluster)

## Calcul de l'importance relative des variables utilisées pour le clustering

- Mesure l’importance relative des variables en comparant leurs moyennes par cluster à la moyenne globale.
- Utilise l’écart-type pour rendre les différences comparables entre variables.
- Normalise les scores pour les exprimer en proportions.

Utilisation : Identifier quelles variables différencient le plus les clusters dans une segmentation.

In [ ]:
# Fonction pour calculer l'importance relative des variables dans le clustering
calculate_importance <- function(data, variables) {
  importance_scores <- sapply(variables, function(var) {
    overall_mean <- mean(data[[var]], na.rm = TRUE)
    cluster_means <- tapply(data[[var]], data$cluster, mean, na.rm = TRUE)
    abs_diff <- abs(cluster_means - overall_mean) / sd(data[[var]], na.rm = TRUE)
    return(mean(abs_diff, na.rm = TRUE))
  })
  importance_scores <- importance_scores / sum(importance_scores)  # Normaliser
  return(importance_scores)
}

In [ ]:
# Calcul du score d'importance relatif pour chaque variable
importance_scores <- calculate_importance(dfUsedForClustering, variablesUsedForClustering)
print(importance_scores)

## Décrire les personas

Calcule les moyennes et écarts-types de chaque variable par cluster

In [ ]:
# Fonction pour décrire les personas en affichant les moyennes et écarts-types par cluster
# (La variable "cluster" doit déjà être présente et factorisée dans data)
describe_personas <- function(data, variables) {
  persona_summary <- data %>%
    group_by(cluster) %>%
    summarise(across(all_of(variables), list(mean = ~mean(.x, na.rm = TRUE),
                                               sd = ~sd(.x, na.rm = TRUE))))
  print(persona_summary)
}

In [ ]:
# Afficher les statistiques descriptives par cluster
describe_personas(dfUsedForClustering, variablesUsedForClustering)

## Identification des variables distinctives

Pour chaque cluster, identifier les variables qui le distingue des autres clusters.

1. Calculer les moyennes globlale et par cluster de chaque variable
1. Associer une couleur (rouge, vert, noir) à chaque variable de chaque cluster en fonction de la capacité de la variable à distinguer le cluster
1. Afficher sous forme de graphique la différence entre la moyenne du cluster et la moyenne globale pour chaque variable de chaque cluster. 

Dans les graphiques, les variables distinctives seront colorées en 

- rouge si leur moyenne dans le cluster est significativement sous la moyenne globale. On dira que leur absence caractérise le cluster.
- vert si leur moyenne est significativement au-dessus de la moyenne globale. On dira que leur présence caractérise le cluster.

In [ ]:
# Initialiser une liste pour stocker tous les prompts
# Initialiser une liste pour stocker tous les prompts
prompts <- list()
all_data_viz <- data.frame()

for (cluster_value in levels(dfUsedForClustering$cluster)) {
  # Extraire les données du cluster
  cluster_data <- dfUsedForClustering %>% filter(cluster == cluster_value)
  
  # Calculer les moyennes par variable pour le cluster et globalement
  means <- sapply(variablesUsedForClustering, function(var) mean(cluster_data[[var]], na.rm = TRUE))
  overall_means <- sapply(variablesUsedForClustering, function(var) mean(dfUsedForClustering[[var]], na.rm = TRUE))
  
  # Identification des caractéristiques marquantes
  characteristic_vars <- sapply(variablesUsedForClustering, function(var) {
    if (all(dfUsedForClustering[[var]] %in% c(0, 1))) {
      prop_cluster <- mean(cluster_data[[var]], na.rm = TRUE)
      prop_overall <- mean(dfUsedForClustering[[var]], na.rm = TRUE)
      p_value <- prop.test(x = c(sum(cluster_data[[var]]), sum(dfUsedForClustering[[var]])),
                           n = c(nrow(cluster_data), nrow(dfUsedForClustering)))$p.value
      prop_diff <- abs(prop_cluster - prop_overall)
      if (p_value < 0.05 && prop_diff > 0.3) {
        return(ifelse(prop_cluster > prop_overall, "green", "red"))
      } else {
        return("black")
      }
    } else {
      t_test <- t.test(cluster_data[[var]], dfUsedForClustering[[var]])
      effect_size <- abs(t_test$estimate[1] - t_test$estimate[2]) / sd(dfUsedForClustering[[var]], na.rm = TRUE)
      if (t_test$p.value < 0.05 && effect_size > 0.25) {
        return(ifelse(t_test$estimate[1] > t_test$estimate[2], "green", "red"))
      } else {
        return("black")
      }
    }
  })
  
  # Préparer les données pour le graphique (si besoin)
  data_viz <- data.frame(
    Variable = variablesUsedForClustering, 
    Mean = means, 
    Overall = overall_means, 
    Cluster = cluster_value,
    Color = characteristic_vars,
    Persona = paste("Persona", cluster_value)
  )
  all_data_viz <- rbind(all_data_viz, data_viz)
}


In [ ]:
all_data_viz_signif <- all_data_viz %>%
  filter(Color %in% c("green", "red"))

ggplot(all_data_viz_signif, aes(x = reorder_within(Variable, Mean, Persona), 
                                y = Mean, fill = Color)) +
  geom_bar(stat = "identity", color = "black", show.legend = FALSE) +
  scale_fill_identity() +
  labs(title = "Visualisation des personas (Variables significatives)", x = "", y = "Moyenne / Proportion") +
  theme_minimal() +
  coord_flip() +
  facet_wrap(~ Persona, scales = "free_y") +
  scale_x_reordered()